In [1]:
import pandas as pd

df = pd.read_feather("train.feather")
print(df.shape)
df.head()

(607844, 23)


,query,url_page,title,source,alt,src,crossorigin,height,ismap,loading,...,srcset,width,class,style,tree_path,deg,text_tag,text,id,is_relevant
0,011103093,https://bank-routing.org/011103093-td-bank-na-...,Routing Number 011103093 - TD Bank NA in LEWIS...,bank-routing.org,Bank Routing,https://bank-routing.org/img/logo.png,None,None,None,None,...,None,None,None,None,"[a, div, nav, header, body, html]",2.0,div,Blog Contacts,3f5e067733178cc0f4b3a97cfc9c62e1,0
1,011103093,https://bank-routing.org/011103093-td-bank-na-...,Routing Number 011103093 - TD Bank NA in LEWIS...,bank-routing.org,011103093 routing number on TD Bank NA check,https://bank-routing.org/img/routing_numbers/r...,None,None,None,None,...,None,None,img-fluid,None,"[div, div, div, main, main, body, html]",1.0,div,Aside from the SWIFT code which each bank is d...,fe64168fa7afeefa73708113c4bc2730,1
2,011103093,https://bank-routing.org/011103093-td-bank-na-...,Routing Number 011103093 - TD Bank NA in LEWIS...,bank-routing.org,DMCA.com Protection Status,https://images.dmca.com/Badges/dmca_protected_...,None,None,None,None,...,None,None,None,None,"[a, p, div, div, div, footer, body, html]",NaN,None,None,5b5c3b6560ae158bf7ecd09f5ead1841,0
3,011103093,https://bank-routing.org/011103093-td-bank-na-...,Routing Number 011103093 - TD Bank NA in LEWIS...,bank-routing.org,Quantcast,https://pixel.quantserve.com/pixel/p-1meCEwxYU...,None,1,None,None,...,None,1,None,None,"[div, noscript, body, html]",3.0,body,Blog Contacts Bank Routing Numbers -> TD Bank ...,91fe71b49453d9037769b5a99c6f0856,0
4,011103093,https://www.gobankingrates.com/banking/banks/t...,TD Bank Routing Number in Connecticut: 2113705...,gobankingrates.com,Loading...,https://cdn.gobankingrates.com/wp-content/them...,None,None,None,None,...,None,None,None,None,"[span, form, div, div, div, main, body, html]",2.0,form,Sign-Up Now Please enter an email. Please ente...,021b4f809739c93f6ff913ce7389a9c9,0


In [ ]:
# trying out embedding (on local GPU)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device="cuda")
sentences = df.url_page.values[:10000]
embeddings = model.encode(sentences, show_progress_bar=True, convert_to_tensor=True, device=None)
embeddings.shape

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

torch.Size([10000, 384])

In [ ]:
# now trying text preprocessing (since we want to encode URLs among others)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)


removable_parts = "https:// http:// www com org"


def url_preprocessing(x: list[str]) -> np.array:
    import numpy as np
    from nltk.stem import WordNetLemmatizer

    lemmatizer = WordNetLemmatizer()
    if x is None:
        return None
    return " ".join(
        np.unique(
            [lemmatizer.lemmatize(e).lower() for e in x if e not in ("") and len(e) > 2]
        )
    )


tmp = df.url_page.str.split(
    r"""{}""".format(
        "|".join((removable_parts + " " + "/ - \. \? \+ : , ; & _").split())
    ),
    regex=True,
).parallel_apply(url_preprocessing)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
for i in range(30):
    print(df.url_page[i], tmp[i])

https://bank-routing.org/011103093-td-bank-na-routing-number 011103093 bank number routing
https://bank-routing.org/011103093-td-bank-na-routing-number 011103093 bank number routing
https://bank-routing.org/011103093-td-bank-na-routing-number 011103093 bank number routing
https://bank-routing.org/011103093-td-bank-na-routing-number 011103093 bank number routing
https://www.gobankingrates.com/banking/banks/td-bank-routing-number-connecticut/ bank banking connecticut gobankingrates number routing
https://www.gobankingrates.com/banking/banks/td-bank-routing-number-connecticut/ bank banking connecticut gobankingrates number routing
https://www.gobankingrates.com/banking/banks/td-bank-routing-number-connecticut/ bank banking connecticut gobankingrates number routing
https://www.gobankingrates.com/banking/banks/td-bank-routing-number-connecticut/ bank banking connecticut gobankingrates number routing
https://www.gobankingrates.com/banking/banks/td-bank-routing-number-connecticut/ bank bankin

In [ ]:
def url_preprocessing(x: list[str]) -> np.array:
    import numpy as np
    from nltk.stem import WordNetLemmatizer

    lemmatizer = WordNetLemmatizer()
    if x is None:
        return None
    return " ".join(
        np.unique(
            [lemmatizer.lemmatize(e).lower() for e in x if e not in ("") and len(e) > 2]
        )
    )
df.alt.str.split(
    r"""{}""".format(
        "|".join((removable_parts + " " + "/ - \. \? \+ : , ; & _").split())
    ),
    regex=True,
).parallel_apply(url_preprocessing)

0                                              bank routing
1              011103093 routing number on td bank na check
2                                    protection status dmca
3                                                 quantcast
4                                                   loading
                                ...                        
607839                                robin zweibel picture
607840                                                     
607841                                                 None
607842                                robin zweibel picture
607843     2016 tribeca film festival robin zweibel 'the...
Name: alt, Length: 607844, dtype: object

In [ ]:
df.alt[:25].str.split(
    r"""{}""".format(
        "|".join((removable_parts + " " + "/ - \. \? \+ : , ; & _").split())
    ),
    regex=True,
).apply(url_preprocessing)

0                                     bank routing
1     011103093 routing number on td bank na check
2                           protection status dmca
3                                        quantcast
4                                          loading
5                           td bank routing number
6                              the gbr closer icon
7                              untitled design (1)
8                                      close popup
9                                   gobankingrates
10                                         zergnet
11                            gobankingrates staff
12                                      me (maine)
13                                  my bank detail
14                                         td bank
15                                           myfin
16                                           myfin
17           td bank routing number in connecticut
18                                                
19                             

In [ ]:
df["text"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').head(30)

0                                         Blog Contacts
1     Aside from the SWIFT code which each bank is d...
2                                                  None
3     Blog Contacts Bank Routing Numbers -> TD Bank ...
4     Sign-Up Now Please enter an email. Please ente...
5     Banking / Banks 011103093  TD Bank Routing Num...
6     Sending you timely financial stories that you ...
7     Sending you timely financial stories that you ...
8     Sending you timely financial stories that you ...
9                                                  None
10                           Sponsored Links by Zergnet
11                  By GOBankingRates Staff May 4, 2021
12    011103093-TD Bank N.A. Routing Number Details ...
13                                                 None
14                                                  4.1
15    Credit cards Compare Rewards Cash Back Balance...
16                                                 None
17    What is a routing number? A routing transi

In [ ]:
for x in df["alt"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').head(50):
    print(x)

Bank Routing
011103093 routing number on TD Bank NA check
DMCA.com Protection Status
Quantcast
Loading...
TD Bank routing number
The GBR Closer icon
Untitled design (1)
Close popup
GOBankingRates
Zergnet
GOBankingRates Staff
ME (MAINE)
My Bank Detail
TD Bank
myfin.us
myfin.us
TD Bank routing number in Connecticut

close
image
notice
None
None
video-icon
image
image
None
None
None
hand shake
Real-Time Bill Payments
Real-Time Bill Payments
Real-Time Online Bill Payments
None
None
Download ACDSee Pro 4
3 stars


Review: Anker 10k hybrid wireless and portable charger, a stylish and versatile charger for around 30
Wacom introduce affordable graphics tablet for Chromebook
Be wary of Huawei's appalling warranties on their Nano memory cards
ACDSee Pro 3 photo management software review
half star




 alt=
